# Introduction to the R package TDA

##  Abstract

RパッケージのTDAの使い方について短いチュートリアルと紹介をする。TDAパッケージは Topological Data Analysis 向けのいくつかのツールを提供する。特に、以降のような実装を含む。ある与えられたデータについて、元になる空間のトポロジー情報、例えば、距離函数(distance function)、測度(measure)への距離、k近傍法(kNN)密度推定、カーネル密度推定、カーネル距離、を提供する。これらの函数のサブレベル（もしくは、スーパーレベル）の凸なトポロジー特徴は persistent homology で定量化できる。ここでは、Rips filtration の persistent homology の機能と、点群の格子で評価される任意の函数のサブレベル（もしくはスーパーレベル）の persistent homology の機能を含む C++ライブラリのGUDHI, Dionysus, PHAT の効果的なアルゴリズムへのRのI/Fを提供する。永続性ダイアグラムの結果の機能の重要性は？で議論されている手法を実装した函数で分析できる。RのTDAパッケージは、密度のクラスタリング用のアルゴリズムの実装も含む。それは、密度函数に関する確率質量の空間的構成を特定し、系統樹とクラスターツリーによってそれを可視化する。

## 1. Introduction

Topological Data Analysis (TDA) はデータのトポロジー構造を見つける方法の集合だ。コンピュータによるトポロジーの最近の進歩は、データからトポロジーの不変量を実際に計算できるようにした。これらの手順の入力は一般的に点群の形をとり、興味深いトポロジーの特徴がサンプリングの中で失った、未知の低い次元のセットSからのノイズの入った観測の確率とみなされる。出力は、Sのトポロジー機能を推定するのに使われるデータサマリの集合だ。

TDAの１つのアプローチは persistent homology で、同時に複数のスケールでのホモロジーの研究の方法だ。より正確には、フレームワークと、ネストされたトポロジーの空間の集まりのトポロジーの進化を定量化する効果的なアルゴリズムを提供する。与えられた実数の函数 $f$ 、Section 2 で説明するような、について、persistent homology は低いレベルのセット $\{x:f(x) \geq t \}$ (もしくは、大きいレベルのセット $\{x:f(x) \leq t\}$ )のトポロジーが $t$ が $-\infty$ から $\infty$ まで増加（あるいは、$\infty$ から $-\infty$ まで減少)するときに、どのように変化するかを説明する。この情報は、persistenceダイアグラム、平面内の点の複数セット、ある間隔の $t$ で表現されるホモロジー特徴である生成と消滅にそれぞれ対応する。

この論文は、RのTDAパッケージの説明とし、それはC++ライブラリの GUDHI, Dionysus, PHAT の効果的なアルゴリズムへのユーザーフレンドリーなI/Fを提供する。

Section 2では、広く研究されている函数をどのように計算するか説明する。それは、距離関数(distance function, distFct)、測度関数(measure function, dtm)、k近傍法密度推定(k Nearest Neighbor density estimator, knnDE)、カーネル密度推定(kernel density estimator, kde)、そして、カーネル距離(kernel distance, kernelDist) だ。

Section 3は、persistenceダイアグラムの計算とする。関数 gridDiag は点群の格子で評価される関数のサブレベル（もしくはスーパーレベル）のセットの persistent homology の計算ができる。関数 ripsDiag は 点群の上に作られた Rips filtration の persistence ダイアグラムを返す。

persistent homology の鍵となるチャレンジのひとつは、トポロジーのノイズを代表するpersistence diagram のポイントを分離する方法を見つけることだ。persistent homology の統計的な方法はその正確な計算の代案を提供する。おおよその persistence diagrams が計算的に実現不可能な真の diagram に近づくことを高い信頼で知ることは、たいてい実践的な目的に充分だ。？は persistence diagram への信頼できるセットを構築するいくつかの統計的な方法を提案しており、トポロジーのノイズからトポロジーの信号を私達が分離できる他の概要の関数を提案している。方法は TDA パッケージに実装されていて、Section 3 で説明される。

最後に、TDA パッケージは、密度クラスタリングのアルゴリズムの実装を提供する。この方法は、データの生成メカニズムについての特定の知識なしにデータの空間的な組織の特定と可視化ができ、特に、クラスタの数についての先天的な情報なしにできる。Section 4 では、関数 clusterTree について説明し、それは与えられた密度推定、superlevel セットの接続された成分の階層を系統樹、クラスタツリーに変換する。

## 2.距離函数と密度推定

最初のTDAパッケージを使う小さな例題として、ポイントグリッドについて距離函数と密度推定をどのように計算するか示す。設定はTDAでの典型的なものだ。ポイントのセット $X=\{x_1,...,x_n\} \subset \mathbb{R}^d$ はある分散 $P$ で抽出され、データの関数を見つけることで下地の空間のトポロジーの特徴を復元することに関心がある。以下のコードは単位円から400個の点のサンプルを生成し、関数を評価するための点のグリッドを構築する。

```R
library("TDA")
X <- circleUnif(400)
Xlim <- c(-1.6, 1.6); Ylim <- c(-1.7, 1.7);
Xseq <- seq(Xlim[1], Xlim[2], by = by)
Yseq <- seq(Ylim[1], Ylim[2], by = by)
Grid <- expand.grid(Xseq, Yseq)
```

TDAパッケージは、以下の関数の実装を提供する。

- 距離函数は、それぞれの$y \in \mathbb{R}^d $ を$ \Delta (y) = inf_{x \in X} \parallel x-y \parallel_2 $のように定義し、グリッドのそれぞれの点を以下のコードで計算する。

```R
distance <- distFct(X = X, Grid = Grid)
```

- 与えられた密度の測度 $P$ 、測度への距離 (DTM, distance to measure) は、それぞれの $y \in \mathbb{R}^d$ を$$ d_{m_0} (y) = \sqrt{\frac{1}{m_0} \int_0^{m_0} (G_y^{-1} (u))^2 du} $$ と定義される。
